# Case 1: Group Life Insurance

Actuarial Science Undergratuate Program | Business Faculty | Risk Theory Class | José Enrique Pérez Salvador

A small Mexican manufacturing business has a 1-year group life insurance contract on its 14 permanent employees. Your actuarial manager, María, working for the Mexican insurer that provides the group life insurance to the manufacturing business, has the following assumptions when modeling the contract:

1) The mortality table “Tasas de Mortalidad Grupo CNSF 2000-G (1991-1998)” (download it from https://mort.soa.org/  ).
2) Each employee is insured for the amount of his/her salary rounded up to the next 1,000 MXN.
3) The group’s data are given in the next table.

You are an actuarial analyst who reports to María.

...

## h) María requires you a risk analysis to answer the question: What are the chances that the insurer will lose money in this contract❓ Use the following approximations:... 1 million simulations

In [ ]:
* Number of simulations;
%let m = 1000000;
* Gross premium;
%let G = 2947.72;

In [ ]:
proc iml;
	* Fix the random seed to replicate results between users and accross time;
	call randseed(2025);	
	* Number of risk units in the individual risk model;
	n=14;
	* Matrix for the parameters;
	par=j(n,2,.);
	* Benefits;
	par[,1]={15000,16000,20000,28000,31000,18000,26000,24000,60000,14000,17000,19000,30000,55000};
	* Mortality rates;
	par[,2]={0.000965,0.001069,0.001252,0.001434,0.001505,0.003751,0.004037,0.004698,0.017778,0.000884,0.001031,0.001201,0.002078,0.007682};
	* Matrix for the Bernoulli simulations;
	ber = j(n,&m.,.);
	* Matrix for the loss simulations;
	X = j(n,&m.,.);
	print(par);
	* Simulating the deaths of the 14 employees;
	do i=1 to n;
		aux=j(1,&m.,.);
		q = par[i,2];
		call randgen(aux,"Bernoulli",q);
		ber[i,]=aux;
	end;
	* Simulating the losses; 
	X = par[,1]#ber;
	S = X[+,];
	* Sending the results to a dataset;
	create aggclaim var{S};
	append;
	close aggclaim;
quit;


In [ ]:
* Changing formats and labels so the data set looks better;
proc datasets lib=work nodetails nolist;
	modify aggclaim;
	label
	S="Total claim in the group life insurance"; 
	format
	S nlnum16.2;
quit;

In [ ]:
title "Distribution of the aggregate loss (S)";
proc sgplot data=work.aggclaim;
	histogram S;
	density S / type=normal transparency=0.7;
	xaxis grid;
	yaxis grid;
run;
title;

In [ ]:
proc freq data=work.aggclaim;
run;

In [ ]:
title "Are the simulated mean, variance and skewness similar to the previous results?";
proc means data=work.aggclaim mean var skewness;
	var S;
run;
title;

In [ ]:
title "Probability of losing money in this contract";
proc sql;
	select count(*)/&m. as "P[S>G]"n
	from work.aggclaim
	where S > &G.;
quit;
title;

...

## j) Calculate the exact distribution of 𝑆 with the following approach:
- Define every state of the world i.e. all the possible combinations with an alive/dead status for every employee, how many states of the world are there❓Calculate the total claim and the probability of every state of the world. Calculate the probability function of 𝑆.

In [ ]:
* Number of risk units in the individual risk model;
%let ne=14;
%put &=ne.;
* Gross premium;
%let G = 2947.72;

In [ ]:
%MACRO allcombinations(n=);
/*
Purpose: To calculate all the states of the world for each employee 
n: number of employees
*/
	* i is the number of deaths;
	%do i=1 %to &n.;
		* Computing the number of combinations;
		data _null_;
			call symputx('nc',comb(&n.,&i.));
		run;
		%put There are &nc. state(s) of the world with &i. death(s) in the population of &n. employees.;
		* Generating the combinations;
		proc plan seed=2025;
			factors iter=&nc. ordered id_employee = &i. of &n. comb / noprint;
			output out=comb_&i.;
		run;
		* Saving the results;
		data id_comb_&i.;			
			number_deaths=&i.;
			is_dead=1;
			set comb_&i.;
		run;
	%end;
	* Appending the results;
	data death;
		set id_comb_:;
	run;
	* Delete unuseful data sets;
	proc datasets lib=work nolist;
		delete id_comb_: comb_:;
	run;
	* i is the number of survivors;
	%do i=1 %to &n.;
		* Computing the number of combinations;
		data _null_;
			call symputx('nc',comb(&n.,&i.));
		run;
        %put There are &nc. state(s) of the world with &i. survivors(s) in the population of &n. employees.;
		* Generating the combinations;
		proc plan seed=2025;
			factors iter=&nc. ordered id_employee = &i. of &n. comb / noprint;
			output out=comb_&i.;
		run;
		* Saving the results;
		data id_comb_&i.;			
			number_deaths=&n.-&i.;
			is_dead=0;
			set comb_&i.;
			iter=&nc.-iter+1;
		run;
	%end;
	* Appending the results;
	data alive;
		set id_comb_:;
	run;
	* Delete unuseful data sets;
	proc datasets lib=work nolist;
		delete id_comb_: comb_:;
	run;
	* Appending the deaths and alives employees for each state of the world;
	data comb;
		set death alive;
	run;
	* Ordering the data set to understand better the states of the world;
	proc sort data=comb;
		by number_deaths iter;
	run;
	* Delete unuseful data sets;
	proc datasets lib=work nolist;
		delete death alive;
	run;
%MEND;

In [ ]:
* Execute the macro;
%allcombinations(n=&ne.);

The previous process produces the following data set:

In [ ]:
options obs=100;
proc print data=work.comb noobs;
run;

Create the matrix of the parameters for each employee

In [ ]:
proc iml;
	
	* Number of risk units in the individual risk model;
	n=&ne.;
	* Matrix for the parameters;
	par=j(n,4,.);
	* ID employee;
	par[,1]=t(do(1,n,1));
	* Benefits;
	par[,2]={15000,16000,20000,28000,31000,18000,26000,24000,60000,14000,17000,19000,30000,55000};
	*par[,2]={15000,16000,20000};
	* Mortality rates;
	par[,3]={0.000965,0.001069,0.001252,0.001434,0.001505,0.003751,0.004037,0.004698,0.017778,0.000884,0.001031,0.001201,0.002078,0.007682};
	*par[,3]={0.000965,0.001069,0.001252};
	* Survival rates;
	par[,4]=1-par[,3];
	print par;
	varNames={"id_employee","benefit","q","p"};
	* Sending the results to a dataset;
	create par from par[colname=varNames];
	append from par;
	close par;	
quit;

Joining the states of the world with the parameters

In [ ]:
proc sql;
	create table full_comb as
	select
	b.*
	, case is_dead
	when 1 then benefit
	when 0 then 0 
	else 0 end as claim
	, case is_dead
	when 1 then q
	when 0 then p 
	else 0 end as prob
	from work.par a right join work.comb b on (a.id_employee = b.id_employee)
	order by number_deaths, iter
	; 
quit;

The previous process produces the following data set:

In [ ]:
options obs=100;
proc print data=work.full_comb noobs;
run;

Computing the total claim and its probability for each state of the world

In [ ]:
proc sql;
	create table full_comb2 as
	select
	number_deaths
	, iter
	, sum(claim) as S
	, EXP(SUM(LOG(prob))) as prob
	from full_comb
	group by number_deaths, iter
	;
quit;

The previous process produces the following data set:

In [ ]:
options obs=100;
proc print data=work.full_comb2 noobs;
run;

Grouping by total claim and its probability

In [ ]:
proc sql;
	create table full_comb3 as
	select
	S format nlnum16.2 label="Total claim in the group life insurance"
	, sum(prob) as prob format 16.6 label="Probability"
	from full_comb2
	group by S
	order by S
	;
quit;

The previous process produces the following data set:

In [ ]:
title "Distribution of the aggregate loss (S)";
proc print data=full_comb3 noobs;
run;
title;

In [ ]:
title "Validation of the Distribution of the aggregate loss (S)";
proc means data=full_comb2 sum min max;
	var prob;
run;
title;

In [ ]:
title "Distribution of the aggregate loss (S)";
proc sgplot data=full_comb3;
	needle x=S y=prob;
run;
title;

In [ ]:
title "Probability of losing money in this contract";
proc sql;
	select sum(prob) as "P[S>G]"n
	from full_comb3
	where S > &G.;
quit;
title;

## 🤷🏼 Which approximation would you choose❓ Why❓